# 範例 : (Kaggle)鐵達尼生存預測

# [教學目標]
- 以下用鐵達尼生存預測資料, 觀察計數編碼與特徵雜湊的效果

# [範例重點]
- 了解計數編碼的寫作方式(In[5], Out[5]), 以及計數編碼搭配邏輯斯迴歸對於測結果有什麼影響 (In[7], Out[7]) 
- 觀察 雜湊編碼, 以及 計數編碼+雜湊編碼 分別搭配邏輯斯迴歸對於測結果有什麼影響 (In[8], Out[8], In[9], Out[9])

In [1]:
import pandas as pd
import numpy as np
import copy,time
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

data_path = 'C:/Users/francis/Machine_Learning/francis/Documents/GitHub/4th-ML100Days/data/'
df_train = pd.read_csv(data_path + 'titanic_train.csv')
df_test = pd.read_csv(data_path + 'titanic_test.csv')

Y_train = df_train['Survived']
ids = df_test['PassengerId']
df_train = df_train.drop(['Survived','PassengerId'], axis = 1)
df_test = df_test.drop(['PassengerId'], axis = 1)
df = pd.concat([df_train,df_test], axis = 0)
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [2]:
object_features = []
for dtype,feature in zip(df.dtypes,df.columns):
    if dtype == 'object':
        object_features.append(feature)
print(f'{len(object_features)} Object features : {object_features}')

df = df[object_features]
train_num = Y_train.shape[0]
df = df.fillna('None')
df.head()
print(df.nunique())
print(df)

5 Object features : ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']
Name        1307
Sex            2
Ticket       929
Cabin        187
Embarked       4
dtype: int64
                                                  Name     Sex  \
0                              Braund, Mr. Owen Harris    male   
1    Cumings, Mrs. John Bradley (Florence Briggs Th...  female   
2                               Heikkinen, Miss. Laina  female   
3         Futrelle, Mrs. Jacques Heath (Lily May Peel)  female   
4                             Allen, Mr. William Henry    male   
..                                                 ...     ...   
413                                 Spector, Mr. Woolf    male   
414                       Oliva y Ocana, Dona. Fermina  female   
415                       Saether, Mr. Simon Sivertsen    male   
416                                Ware, Mr. Frederick    male   
417                           Peter, Master. Michael J    male   

                 Ticket Cabin Embarked  
0

In [9]:
# 觀察欄位相異值數量
df.select_dtypes(include = 'object').apply(pd.Series.nunique)

Name        1307
Sex            2
Ticket       929
Cabin        187
Embarked       4
dtype: int64

In [11]:
# 對照組 : 標籤編碼 + 邏輯斯迴歸
df_temp = pd.DataFrame()
for c in df.columns:
    df_temp[c] = LabelEncoder().fit_transform(df[c])
X_train = df_temp[:train_num]
estimator = LogisticRegression()
print(cross_val_score(estimator,X_train,Y_train,cv = 5).mean())
df_temp.head()

0.780004837244799


C:\Users\francis\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\francis\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\francis\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\francis\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\francis\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to

,Name,Sex,Ticket,Cabin,Embarked
0,155,1,720,185,3
1,286,0,816,106,0
2,523,0,914,185,3
3,422,0,65,70,3
4,22,1,649,185,3


In [20]:
# 加上 'Ticket' 欄位的計數編碼
# 第一行 : df.groupby(['Ticket']) 會輸出 df 以 'Ticket' 群聚後的結果, 但因為群聚一類只會有一個值, 因此必須要定義運算
# 例如 df.groupby(['Ticket']).size(), 但欄位名稱會變成 size, 要取別名就需要用語法 df.groupby(['Ticket']).agg({'Ticket_Count':'size'})
# 這樣出來的計數欄位名稱會叫做 'Ticket_Count', 因為這樣群聚起來的 'Ticket' 是 index, 所以需要 reset_index() 轉成一欄
# 因此第一行的欄位, 在第三行按照 'Ticket_Count' 排序後, 最後的 DataFrame 輸出如 Out[5]
#算出一種Ticket會有幾個人擁有
count_df = df.groupby(['Ticket'])['Name'].agg({'Ticket_Count' : 'size'}).reset_index()
# 但是上面資料表結果只是 'Ticket' 名稱對應的次數, 要做計數編碼還需要第二行 : 將上表結果與原表格 merge, 合併於 'Ticket' 欄位
# 使用 how='left' 是完全保留原資料表的所有 index 與順序
df = pd.merge(df,count_df,how = 'left',on = 'Ticket')
df.head()
count_df.sort_values(by = 'Ticket_Count' , ascending = False).head(10)

C:\Users\francis\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version. Use                 named aggregation instead.

    >>> grouper.agg(name_1=func_1, name_2=func_2)

  import sys


,Ticket,Ticket_Count
778,CA. 2343,11
104,1601,8
775,CA 2144,8
335,3101295,7
454,347077,7
459,347082,7
847,S.O.C. 14879,7
824,PC 17608,7
123,19950,6
49,113781,6


In [21]:
# 印出來看看, 加了計數編碼的資料表 df 有何不同
df.head()

,Name,Sex,Ticket,Cabin,Embarked,Ticket_Count_x,Ticket_Count_y,Ticket_Count
0,"Braund, Mr. Owen Harris",male,A/5 21171,None,S,1,1,1
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,PC 17599,C85,C,2,2,2
2,"Heikkinen, Miss. Laina",female,STON/O2. 3101282,None,S,1,1,1
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,113803,C123,S,2,2,2
4,"Allen, Mr. William Henry",male,373450,None,S,1,1,1


In [24]:
# 'Ticket'計數編碼 + 邏輯斯迴歸
df_temp_1 = pd.DataFrame()
for col in object_features:
    df_temp_1[col] = LabelEncoder().fit_transform(df[col])
df_temp_1['Ticket_Count'] = df['Ticket_Count']
X_train = df_temp_1[:train_num]
estimator = LogisticRegression()
print(cross_val_score(estimator,X_train,Y_train,cv = 5).mean())
df_temp_1.head()

0.7811221556805532


C:\Users\francis\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\francis\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\francis\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\francis\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\francis\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to

,Name,Sex,Ticket,Cabin,Embarked,Ticket_Count
0,155,1,720,185,3,1
1,286,0,816,106,0,2
2,523,0,914,185,3,1
3,422,0,65,70,3,2
4,22,1,649,185,3,1


In [29]:
# 'Ticket'計數編碼 + 'Ticket'特徵雜湊 + 邏輯斯迴歸
df_temp_2 = pd.DataFrame()
for col in object_features:
    df_temp_2[col] = LabelEncoder().fit_transform(df[col])
df_temp_2['Ticket_Hash'] = df['Ticket'].map(lambda x : hash(x) % 10)
df_temp_2.head()




'''''''''
df_temp_2['Ticket_Count'] = df['Ticket_Count'].map(lambda x: hash(x) % 10)
X_train = df_temp_2[:train_num]
estimator = LogisticRegression()
print(cross_val_score(estimator,X_train,Y_train,cv = 5).mean())
\

SyntaxError: EOF while scanning triple-quoted string literal (<ipython-input-29-1a0285adb07a>, line 16)

In [32]:
df_temp_2 = pd.DataFrame()
for col in object_features:
    df_temp_2[col] = LabelEncoder().fit_transform(df[col])
df_temp_2['Ticket_Hash'] = df['Ticket'].map(lambda x : hash(x) % 10)
df_temp_2['Ticket_Count'] = df['Ticket_Count']
X_train = df_temp_2[:train_num]
estimator = LogisticRegression()
print(cross_val_score(estimator,X_train,Y_train,cv = 5).mean())
df_temp_2.head()

0.7777513691636992


C:\Users\francis\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\francis\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\francis\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\francis\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\francis\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to

,Name,Sex,Ticket,Cabin,Embarked,Ticket_Hash,Ticket_Count
0,155,1,720,185,3,0,1
1,286,0,816,106,0,6,2
2,523,0,914,185,3,6,1
3,422,0,65,70,3,9,2
4,22,1,649,185,3,0,1


In [ ]:
# 加上 'Cabin' 欄位的計數編碼


In [36]:
# 加上 'Cabin' 欄位的計數編碼


,Name,Sex,Ticket,Cabin,Embarked,Ticket_Count_x,Ticket_Count_y,Ticket_Count
0,"Braund, Mr. Owen Harris",male,A/5 21171,None,S,1,1,1
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,PC 17599,C85,C,2,2,2
2,"Heikkinen, Miss. Laina",female,STON/O2. 3101282,None,S,1,1,1
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,113803,C123,S,2,2,2
4,"Allen, Mr. William Henry",male,373450,None,S,1,1,1
...,...,...,...,...,...,...,...,...
1304,"Spector, Mr. Woolf",male,A.5. 3236,None,S,1,1,1
1305,"Oliva y Ocana, Dona. Fermina",female,PC 17758,C105,C,3,3,3
1306,"Saether, Mr. Simon Sivertsen",male,SOTON/O.Q. 3101262,None,S,1,1,1
1307,"Ware, Mr. Frederick",male,359309,None,S,1,1,1


# 作業1
* 參考範例，將鐵達尼的艙位代碼( 'Cabin' )欄位使用特徵雜湊 / 標籤編碼 / 計數編碼三種轉換後，  
與其他類別型欄位一起預估生存機率

# 作業2
* 承上題，三者比較效果何者最好?

In [65]:
import pandas as pd
import numpy as np
import copy,time
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

data_path = 'C:/Users/francis/Machine_Learning/francis/Documents/GitHub/4th-ML100Days/data/'
df_train = pd.read_csv(data_path + 'titanic_train.csv')
df_test = pd.read_csv(data_path + 'titanic_test.csv')

Y_train = df_train['Survived']
ids = df_test['PassengerId']
df_train = df_train.drop(['Survived','PassengerId'], axis = 1)
df_test = df_test.drop(['PassengerId'], axis = 1)
df = pd.concat([df_train,df_test], axis = 0)
df.head()
object_features = []
for dtype,feature in zip(df.dtypes,df.columns):
    if dtype == 'object':
        object_features.append(feature)
print(f'{len(object_features)} Object features : {object_features}')

df = df[object_features]
train_num = Y_train.shape[0]
df = df.fillna('None')
df.head()
print(df.nunique())
print(df)

5 Object features : ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']
Name        1307
Sex            2
Ticket       929
Cabin        187
Embarked       4
dtype: int64
                                                  Name     Sex  \
0                              Braund, Mr. Owen Harris    male   
1    Cumings, Mrs. John Bradley (Florence Briggs Th...  female   
2                               Heikkinen, Miss. Laina  female   
3         Futrelle, Mrs. Jacques Heath (Lily May Peel)  female   
4                             Allen, Mr. William Henry    male   
..                                                 ...     ...   
413                                 Spector, Mr. Woolf    male   
414                       Oliva y Ocana, Dona. Fermina  female   
415                       Saether, Mr. Simon Sivertsen    male   
416                                Ware, Mr. Frederick    male   
417                           Peter, Master. Michael J    male   

                 Ticket Cabin Embarked  
0

In [66]:
df.select_dtypes(include = 'object').apply(pd.Series.nunique)

Name        1307
Sex            2
Ticket       929
Cabin        187
Embarked       4
dtype: int64

In [67]:
# 加上 'Cabin' 欄位的計數編碼
cabin_count = df.groupby(['Cabin'])['Name'].agg({'Cabin_Count' : 'size' }).reset_index()
df = pd.merge(df,cabin_count,how = 'left' , on = 'Cabin')
df.head()

C:\Users\francis\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version. Use                 named aggregation instead.

    >>> grouper.agg(name_1=func_1, name_2=func_2)

  


,Name,Sex,Ticket,Cabin,Embarked,Cabin_Count
0,"Braund, Mr. Owen Harris",male,A/5 21171,None,S,1014
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,PC 17599,C85,C,2
2,"Heikkinen, Miss. Laina",female,STON/O2. 3101282,None,S,1014
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,113803,C123,S,2
4,"Allen, Mr. William Henry",male,373450,None,S,1014


In [73]:
cabin_count.sort_values(by = 'Cabin_Count', ascending = 'False')

,Cabin,Cabin_Count
0,A10,1
134,D38,1
130,D34,1
125,D22,1
119,D11,1
...,...,...
117,D,4
47,B57 B59 B63 B66,5
184,G6,5
80,C23 C25 C27,6


In [74]:
# 'Cabin'計數編碼 + 邏輯斯迴歸
df_temp = pd.DataFrame()
for col in object_features:
    df_temp[col] = LabelEncoder().fit_transform(df[col])
df_temp['Cabin_Count'] = df['Cabin_Count']
X_train = df_temp[:train_num]
estimator = LogisticRegression()
print(cross_val_score(estimator,X_train,Y_train,cv = 5).mean())

0.7856230275549181


C:\Users\francis\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\francis\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\francis\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\francis\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\francis\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to

In [77]:
# 'Cabin'特徵雜湊 + 邏輯斯迴歸
df_temp_1 = pd.DataFrame()
for col in object_features:
    df_temp_1[col] = LabelEncoder().fit_transform(df[col])
df_temp_1['Cabin_Hash'] = df['Cabin'].map(lambda x : hash(x) % 10)
X_train = df_temp_1[:train_num]
estimator = LogisticRegression()
print(cross_val_score(estimator,X_train,Y_train,cv = 5).mean())

0.7766528819375365


C:\Users\francis\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\francis\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\francis\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\francis\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\francis\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to

In [78]:
# 'Cabin'計數編碼 + 'Cabin'特徵雜湊 + 邏輯斯迴歸
df_temp_2 = pd.DataFrame()
for col in object_features:
    df_temp_2[col] = LabelEncoder().fit_transform(df[col])
df_temp_2['Cabin_Count'] = df['Cabin_Count']
df_temp_2['Cabin_Hash'] = df['Cabin'].map(lambda x :hash(x) % 10)
X_train = df_temp_2[:train_num]
estimator = LogisticRegression()
print(cross_val_score(estimator,X_train,Y_train,cv = 5).mean())

0.784474252842502


C:\Users\francis\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\francis\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\francis\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\francis\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\francis\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to